In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Installing the Packages :

In [2]:
!pip install -q langchain
!pip install -q langchain_community
!pip install -q sentence_transformers
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q ctransformers
!pip install -q pypdf
!pip install -q faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9

In [3]:
!pip install -q --upgrade langchain

In [4]:
!pip install -q sentence-transformers

In [5]:
!pip install -q langchain pypdf

In [6]:
import os
import warnings
warnings.filterwarnings('ignore')

#Importing LLM

In [7]:
from langchain.llms import CTransformers

In [8]:
model_path = "/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin"

In [9]:
llm = CTransformers(
    model = model_path,
    model_type = "llama",
    config = {'max_new_tokens': 600, 'temperature': 0.01, 'context_length': 5000}
)

#Creating embeddings

In [10]:
from sentence_transformers import SentenceTransformer

In [11]:
embedding_model = SentenceTransformer('BAAI/bge-base-en-v1.5', cache_folder='.')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

In [13]:
embedding_model_path = "/content/models--BAAI--bge-base-en-v1.5/snapshots/a5beb1e3e68b9ab74eb54cfd186867f64f240e1a"

In [14]:
embeddings = HuggingFaceEmbeddings(model_name = embedding_model_path,
                                   model_kwargs = {'device':'cuda'},
                                  encode_kwargs = {'normalize_embeddings':True}
                                   )

<ipython-input-14-41e16a778cc7>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = embedding_model_path,


#Loading the PDF

In [15]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
pdf_reader = PyPDFLoader('/content/drive/MyDrive/fibromyalgia-information-booklet-july2021.pdf')

In [17]:
document = pdf_reader.load()

#Creating chunks from PDF

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                               chunk_overlap = 200,
                                               length_function = len)

In [19]:
chunk = text_splitter.split_documents(document)

#Passing chunks and embeddings to vectorstore

In [20]:
from langchain.vectorstores import FAISS

In [21]:
vector_db = FAISS.from_documents(documents = chunk, embedding = embeddings)

#Creating a retriever

In [22]:
retriever = vector_db.as_retriever()

#Initializing a template

In [23]:
template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Chat History:
{chat_history}
Follow up Input: {question}
Standalone questions:"""

In [24]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

In [25]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

In [26]:
chain = ConversationalRetrievalChain.from_llm(llm = llm, retriever = retriever,  condense_question_prompt = CONDENSE_QUESTION_PROMPT,
                                              return_source_documents = True, verbose=False)

#Asking Query from User

In [27]:
chat_history = []
query = "what are the causes of fibromyalgia?"
result = chain.invoke({"question": query, "chat_history": chat_history})
print(result["answer"])

 Fibromyalgia is a complex condition, and there isn’t a single cause. Research suggests that it may be caused by a combination of genetic, environmental, and lifestyle factors. These can include things like:
• Genetics – some people may be more prone to fibromyalgia due to their genetic makeup.
• Hormonal changes – fluctuations in hormone levels, such as those that occur during menopause or pregnancy, can trigger symptoms.
• Infections – some people may develop fibromyalgia after having a viral or bacterial infection.
• Trauma – physical or emotional trauma, such as a car accident or sexual assault, can lead to the development of fibromyalgia.
• Lifestyle factors – things like stress, lack of sleep, and poor diet can contribute to the development of fibromyalgia.
It’s important to note that fibromyalgia is not caused by overuse or abuse of muscles or joints, as it is sometimes mistakenly believed to be.


In [28]:
chat_history

[]

In [29]:
from langchain.schema import AIMessage, HumanMessage

In [30]:
chat_history.extend(
    [
        HumanMessage(content=query),
        AIMessage(content = result["answer"])
    ]
    )

In [31]:
chat_history

[HumanMessage(content='what are the causes of fibromyalgia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=' Fibromyalgia is a complex condition, and there isn’t a single cause. Research suggests that it may be caused by a combination of genetic, environmental, and lifestyle factors. These can include things like:\n• Genetics – some people may be more prone to fibromyalgia due to their genetic makeup.\n• Hormonal changes – fluctuations in hormone levels, such as those that occur during menopause or pregnancy, can trigger symptoms.\n• Infections – some people may develop fibromyalgia after having a viral or bacterial infection.\n• Trauma – physical or emotional trauma, such as a car accident or sexual assault, can lead to the development of fibromyalgia.\n• Lifestyle factors – things like stress, lack of sleep, and poor diet can contribute to the development of fibromyalgia.\nIt’s important to note that fibromyalgia is not caused by overuse or abuse of muscles or join

#Deploying in Gradio

In [32]:
pip install gradio langchain sentence-transformers ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 909.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 863.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.0 MB/s eta 0:00:00


In [34]:
import gradio as gr
from langchain.llms import CTransformers
from sentence_transformers import SentenceTransformer

# Load the models
llm = CTransformers(
    model="/content/drive/MyDrive/llama-2-7b-chat.ggmlv3.q4_0.bin",  # Use the correct path to your model
    model_type="llama",
    config={'max_new_tokens': 600, 'temperature': 0.01, 'context_length': 5000}
)
retriever_model = SentenceTransformer('all-MiniLM-L6-v2')

# Define the inference function
def rag_inference(query):
    # Embedding the query using SentenceTransformer
    query_embedding = retriever_model.encode(query)

    # Using the LLaMA model for text generation based on the query
    response = llm(query)

    return response

# Gradio interface
interface = gr.Interface(
    fn=rag_inference,
    inputs="text",  # Input type is text
    outputs="text", # Output is also text
    title="Medical Chatbot",
    description="Ask a question, and the model will generate an answer using RAG."
)

# Launch the Gradio app
interface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ae0419d5d87af0abc4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
